In [6]:
import os
import json
import re
import pandas as pd

def parse_jsonl(data):
    parsed_data = []
    for line in data:
        parsed_data.append(json.loads(line))

    return parsed_data

def get_main_df(path, file_in_question):

    main_df = pd.DataFrame()
    for root, dirs, files in os.walk(path):
        if "layer" not in root:
            continue

        for file in files:

            if file == file_in_question:
                with open(os.path.join(root, file)) as f:
                    data = f.readlines()


                parsed_data = parse_jsonl(data)
                df = pd.DataFrame(parsed_data)
                
                # We have tp, fp, tn, fn
                # We want precision, recall, f1
                df["precision"] = df["tp"] / (df["tp"] + df["fp"])
                df["recall"] = df["tp"] / (df["tp"] + df["fn"])
                df["f1"] = 2 * (df["precision"] * df["recall"]) / (df["precision"] + df["recall"])
                df["accuracy"] = (df["tp"] + df["tn"]) / (df["tp"] + df["tn"] + df["fp"] + df["fn"])
                df["layer"] = re.search(r"layer(\d+)", root).group(1)

                main_df = pd.concat([main_df, df])

    return main_df

def get_best(main_df):
    best_accuracy = main_df[main_df["accuracy"] == main_df["accuracy"].max()]["accuracy"]
    best_f1 = main_df[main_df["f1"] == main_df["f1"].max()]["f1"]
    layer = main_df[main_df["f1"] == main_df["f1"].max()]["layer"]
    print(f"Best layer: {layer.values[0]}")
    return best_accuracy, best_f1, main_df[main_df["f1"] == main_df["f1"].max()]


path = "gpt2_humaneval"
file_in_question = "random_forest.jsonl"
main_df = get_main_df(path, file_in_question)
best_accuracy, best_f1, best_restult = get_best(main_df)
print(f"For {file_in_question} in {path}")
print(f"BEST F1: {best_f1}")
print(f"BEST ACCURACY: {best_accuracy}")
best_restult

Best layer: 0
For random_forest.jsonl in gpt2_humaneval
BEST F1: 13    0.638889
Name: f1, dtype: float64
BEST ACCURACY: 13    0.606061
Name: accuracy, dtype: float64


,classifier,n_features,tp,fp,tn,fn,y_pred,y_test,best_params,dataset,precision,recall,f1,accuracy,layer
13,random_forest,2000,23,16,17,10,"[0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, ...","[1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, ...","{'max_depth': 50, 'max_features': 'sqrt', 'min...",gpt2_humaneval/layer0/,0.589744,0.69697,0.638889,0.606061,0


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set seaborn style for better aesthetics
sns.set_style("whitegrid")

from dataclasses import dataclass

@dataclass
class Metric:
    name: str
    label: str

F1 = Metric("f1", "F1 Score")
ACCURACY = Metric("accuracy", "Accuracy")

ACTIVE_METRIC = F1

# Create the figure
plt.figure(figsize=(8, 6))

# Plot mean F1 score across all layers with confidence interval
sns.lineplot(data=main_df, x="n_features", y=ACTIVE_METRIC.name, ci=95, label=f"Mean {ACTIVE_METRIC.label}", color="black", linestyle="dashed")

# Plot Layer 11
# df_11 = main_df[main_df["layer"] == "11"]
# plt.plot(df_11["n_features"], df_11[ACTIVE_METRIC.name], label="Layer 11", color="red", linestyle="-",alpha=0.3)

# # Plot Layer 1
# df_1 = main_df[main_df["layer"] == "1"]
# plt.plot(df_1["n_features"], df_1[ACTIVE_METRIC.name], label="Layer 1", color="blue", linestyle="-",alpha=0.3)

# Labels and title
plt.xlabel("Number of Features", fontsize=12)
plt.ylabel(ACTIVE_METRIC.label, fontsize=12)
plt.title(f"{ACTIVE_METRIC.label} with Confidence Interval Across Layers", fontsize=14, fontweight="bold")

# Add a legend
plt.legend(fontsize=10)

# make the x log scale
plt.xscale("log")

# Improve layout
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()

#define axis limits
plt.ylim(0, 1)

# Show the plot
plt.show()
